In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langchain_groq import ChatGroq

llamaChatModel = ChatGroq(model="llama-3.3-70b-versatile")

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [3]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [5]:
chain = prompt | llamaChatModel | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

"Here's a curious fact about Cristiano Ronaldo: \n\nCristiano Ronaldo has a hotel chain named after him, but what's interesting is that he has a 'CR7' hotel in Funchal, Madeira, where he was born, and it has a museum dedicated to his football career on the top floor. However, the most curious part is that the hotel has a 'CR7' museum, but he also has a life-size wax replica of himself in the lobby, which is quite an unusual feature for a hotel."

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [7]:
chain = prompt | llamaChatModel.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano '

## Combining LCEL Chains


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | llamaChatModel | StrOutputParser()

In [10]:
chain.invoke("Modi")

'Narendra Modi is the 14th and current Prime Minister of India, serving since 2014 and being re-elected in 2019.'

## Combined Chain

In [13]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | llamaChatModel | StrOutputParser()


In [14]:
composed_chain.invoke({"politician": "rahul Gandhi"})

'Rahul Gandhi, as a prominent Indian politician and a member of the Indian National Congress party, has been involved in various initiatives and policies that have had both positive and negative impacts on humanity. Here are some aspects to consider:\n\nPositive aspects:\n\n1. **Pro-poor and pro-farmer policies**: Rahul Gandhi has consistently advocated for the rights of the poor, farmers, and marginalized communities. He has supported policies like the Minimum Income Guarantee Scheme (NYAY) and the Farmers\' Loan Waiver Scheme, which aim to alleviate poverty and support agricultural development.\n2. **Education and healthcare**: Gandhi has emphasized the importance of education and healthcare in his political campaigns. He has supported initiatives to improve access to quality education and healthcare, particularly for disadvantaged groups.\n3. **Environmental conservation**: Rahul Gandhi has spoken out on environmental issues, such as climate change, deforestation, and pollution. He 

## LCEL chain at work in a typical RAG app


In [20]:
import bs4
from langchain import hub
# from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever()


prompt = ChatPromptTemplate.from_template("""
You are an intelligent assistant. Use the following retrieved context
to answer the user’s question.

Context:
{context}

Question:
{question}

Answer clearly and concisely. If the context does not contain the answer,
say "I don’t know" instead of guessing.
""")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llamaChatModel
    | StrOutputParser()
)

In [21]:
rag_chain.invoke("What is Task Decomposition?")

/opt/anaconda3/envs/llmapp/lib/python3.11/site-packages/langchain_groq/chat_models.py:665: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


'Task decomposition is the process of breaking down a complex task into smaller, simpler steps or subgoals. It can be done through various methods, including using large language models (LLMs) with simple prompting, task-specific instructions, or human inputs.'